#### WIAA ELO-based Model for Measuring Team Strength
Now, we have done about as well as we can attempt to build out an ELO-based model to determine a better system for measuring the strength of teams for the WIAA tournament. However, it may be helpful to work through [an explainer](https://nicidob.github.io/nba_elo/) to figure out how to develop this system.

Classically, ELO only uses wins and losses to measure performance. Initially, every competitor is assigned an initial rating. FiveThirtyEight uses 1300 in their models. Higher ratings = better teams, lower ratings = worse teams. To figure out if team A will beat team B, we compute the difference in scores. To map this difference into probabilities, we use a logistic model that maps a function of the reals onto a closed scale of $[0, 1]$. If the difference in scores is zero, then the probablity of a win or loss is 50%. The closer to 0 or 1 the probability gets, that should be interpreted as a team being more likely to lose or win, respectively. You can determine both team's ability to win a contest by calculating the probablity of one team winning. Then, subtract that probability from 1 to determine the probability of the other team winning.

There is a scale that determines how sharp the logistic curve is, which most ELO variants set the scale to 400. Roughly, 400 means if your team's ELO score is +100 higher than your opponent's, then you win 64% of the time, +200 75% of the time, +400 91% of the time, and so on. Most people familiarwith machine learning have seen a logistic function, $\frac{1}{1+e^{-x}}$, but ELO is written with the base-10 variant. It's basically equivalent, you just need to adjust the scale. For example, 400 in base 10 is 173 in base $e$.

When we observe an outcome, we give the winner some fraction of $K$ points, and take away the same amount of points from the loser. The fraction is based on the loser's probability of winning. The more likely you are to win, the fewer points you get. Equal odds $\rightarrow$ 50% of $K$. If you had a 90% chance of winning and won, 10% of $K$, and vice versa.

#### ELO Formulas
More formally, we have a sigmoid with a scale of 400:

$\sigma(x) = \frac{1}{1+10^{\frac{-x}{400}}}$

We have a probability of A winning as:

$P(A>B) = \sigma(R_A - R_B)$

We will assume A wins, so the error given A won is the probability of B winning:

$E_{(w=A)} = P(B>A)$

The update is just that amount of $K$:

$R_A^{(t+1)} = R_A^{(t)} + KE_{(w=A)}$

$R_B^{(t+1)} = R_B^{(t)} + KE_{(w=B)}$

#### 538's NBA ELO
The FiveThirtyEight model makees three adjustments.
1. After every season, all teams get regressed to the mean.
2. The home team gets an extra 100 points when comparing winning probabilities, as there's home court advantage. Since the home team wins about 60% of the time, this should probably be +70.
3. $K$ is adjusted depending on the margin of victory and the ELO difference. You win by more points, you get more of $K$ with a small adjustment for team strength as well.

It may be useful to utilize some of [FiveThirtyEight's NBA ELO definitions](https://github.com/fivethirtyeight/data/tree/master/nba-elo) to build a simplified model for the WIAA Tournament:

| Header | Definition |
| :----- | :--------- |
| `gameorder` | Play order of every game HSGBB for WIAA teams |
| `game_id` | Unique ID for each game |
| `year_id` | Season ID based on the year which the season ended |
| `date_game` | Date of game played |
| `is_playoffs` | Flag for WIAA playoff games |
| `team_name` | Name of team involved in a game |
| `pts` | Points score by team in that game |
| `elo_i` | Team ELO entering the game |
| `elo_n` | Team ELO following the game |
| `win_equiv` | Equivalent number of wins in a 24-game season for a team of elo_n quality |
| `opp_name` | Team name of opponent played |
| `opp_pts` | Points scored by opponent |
| `opp_elo_i` | Opponent ELO entering the game |
| `opp_elo_n` | Opponent ELO following the game |
| `game_location` | Home (H) or Away (A) *will work on adding neutral (N) later* |
| `game_result` | Win or loss for the team in the team_id column, 0 for loss, 1 for win |
| `forecast` | ELO-based chances of winning for the team in `team_id` colun, based on ELO ratings and location |

The next issue we have to overcome is to recreate a file that is similar to [FiveThirtyEight's NBA ELO file](https://raw.githubusercontent.com/fivethirtyeight/data/master/nba-elo/nbaallelo.csv). I wrote this process using [file_name_here]().